In [1]:
!pip install opensmile praat-parselmouth librosa disvoice

In [2]:
import parselmouth
sound = parselmouth.Sound("/content/harvard.wav")

In [3]:
from parselmouth import Sound
from parselmouth.praat import call
import numpy as np

class FeatureGenerator:
    def __init__(self, sound): # sound = Sound(wav_file)
        self.sample_rate = 16_000
        self.fft_size = 512
        self.window_size = 400
        self.hop_size = 160
        self.time_step = self.hop_size/self.sample_rate
        self.call = parselmouth.praat.call
        self.high_resolution_desired = False


    def _get_wv_feats(self, sound):
        pre_emphasis = call(sound, 'Filter (pre-emphasis)', 80)
        spectrum = call(pre_emphasis, 'To Spectrum', 'yes')
        cog = call(spectrum, 'Get centre of gravity', 2)
        std = call(spectrum, 'Get standard deviation', 2)
        skw = call(spectrum, 'Get skewness', 2)
        kur = call(spectrum, 'Get kurtosis', 2)
        return {'KUR':kur,'SkW':skw,'COG':cog,'SD':std}

    def _get_mfcc(self, sound, start, end):
        """mfcc: pre-emphasis => window => DFT => MelFilterBank => log => IDFT """

        mfcc = sound.to_mfcc(number_of_coefficients=12, time_step=self.time_step, window_length=self.window_size/self.sample_rate, maximum_frequency=7600)
        mfcc_arr = mfcc.to_array()[1:] # 0 index is the energy of cepstrum
        mfcc_bins = mfcc.x_bins()[:, 0]

        def to_frame(time):
            frame = np.searchsorted(mfcc_bins, time)-1
            return frame if frame >= 0 else 0

        start = to_frame(start)
        end = to_frame(end)

        _mfcc = mfcc_arr[:, start:end+1]
        _mfcc = np.mean(_mfcc, axis=-1)
        return _mfcc

    def _get_formant(self, sound, start, end, get_part = True):
        """
        formant
        """
        formant = sound.to_formant_burg(time_step=self.time_step, max_number_of_formants=4.5, maximum_formant=4700.0, window_length=self.window_size/self.sample_rate, pre_emphasis_from=50)
        duration = end - start
        _formant = {}
        for f in range(3):  # f1~f3
            formant_ls = []
            for i in range(3,100,3):
                formant_ls.append(formant.get_value_at_time(formant_number=f+1, time=start + i/100*duration))
            if get_part:
                formant_ls = formant_ls[10:-11]
            formant_name = "f"+str(f+1)
            _formant[formant_name] = formant_ls
        return _formant


    def _get_pitch(self, sound, start, end, get_part = True):
        """f0"""

        duration = end - start
        pitch = sound.to_pitch(time_step=self.time_step, pitch_floor=75.0, pitch_ceiling=600.0)

        _ff = []
        for i in range(3,100,3):
          _ff.append(pitch.get_value_at_time(time=start + i/100*duration))
        if get_part:
          _ff = _ff[10:-11]

        return _ff

    def _get_intensity(self, sound, start, end, get_part = True):
        """intensity"""

        duration = end - start
        intensity = sound.to_intensity(minimum_pitch=100.0, time_step=self.time_step, subtract_mean=True)
        _int = []
        for i in range(3,100,3):
            _int.append(intensity.get_value(time=start + i/100*duration))
        if get_part:
          _int = _int[10:-11]

        return _int

    def _get_jitter(self, sound, start, end):
        """jitter"""
        point_process = self.call(sound, "To PointProcess (periodic, cc)", 75, 600)  # pitch_floor=75, pitch_ceiling=600
        local_jitter = self.call(point_process, "Get jitter (local)", start, end, 0.0001, 0.02, 1.3)
        localabsolute_jitter = self.call(point_process, "Get jitter (local, absolute)", start, end, 0.0001, 0.02, 1.3)
        rap_jitter = self.call(point_process, "Get jitter (rap)", start, end, 0.0001, 0.02, 1.3)
        ppq5_jitter = self.call(point_process, "Get jitter (ppq5)", start, end, 0.0001, 0.02, 1.3)
        ddp_jitter = self.call(point_process, "Get jitter (ddp)", start, end, 0.0001, 0.02, 1.3)

        _jitter = [local_jitter, localabsolute_jitter, rap_jitter, ppq5_jitter, ddp_jitter]

        return _jitter

    def _get_shimmer(self, sound, start, end):
        """shimmer"""
        point_process = self.call(sound, "To PointProcess (periodic, cc)", 75, 600)  # pitch_floor=75, pitch_ceiling=600
        local_shimmer = self.call([sound, point_process], "Get shimmer (local)", start, end, 0.0001, 0.02, 1.3, 1.6)
        localdb_shimmer = self.call([sound, point_process], "Get shimmer (local_dB)", start, end, 0.0001, 0.02, 1.3, 1.6)
        apq3_shimmer = self.call([sound, point_process], "Get shimmer (apq3)", start, end, 0.0001, 0.02, 1.3, 1.6)
        apq5_shimmer = self.call([sound, point_process], "Get shimmer (apq5)", start, end, 0.0001, 0.02, 1.3, 1.6)
        apq11_shimmer = self.call([sound, point_process], "Get shimmer (apq11)", start, end, 0.0001, 0.02, 1.3, 1.6)
        dda_shimmer = self.call([sound, point_process], "Get shimmer (dda)", start, end, 0.0001, 0.02, 1.3, 1.6)
        _shimmer = [local_shimmer, localdb_shimmer, apq3_shimmer, apq5_shimmer, apq11_shimmer, dda_shimmer]

        return _shimmer

    def _get_hnr(self, sound, start, end, method='cc', get_part=True):
        """
        Calculate Harmonics-to-Noise Ratio (HNR); represents the degree of acoustic periodicity and Voice quality
        """
        if method == 'ac':
            hnr= sound.to_harmonicity_ac(time_step=self.time_step) # cross-correlation method (preferred).
        else:
            hnr= sound.to_harmonicity_cc(time_step=self.time_step) # cross-correlation method (preferred).

        duration = end - start
        _hnr= []

        for i in range(3,100,3):
            _hnr.append(hnr.get_value(time=start + i/100*duration))
        if get_part:
          _hnr = _hnr[10:-11]
        _hnr = [h for h in _hnr if h != -200]
        return _hnr

In [4]:
my_feature_generator = FeatureGenerator(sound)
# center of gravity: within a frequency distribution, the point that splits two halves with the same amounts of postive/negative energy/frequencies
# skewness: similar to COG, a tendency on whether frequencies are positive or negative
# kurtosis: noise measurement

wave_features = my_feature_generator._get_wv_feats(sound)
print(wave_features)

{'KUR': 1.5378509206722457, 'SkW': 0.337997181233544, 'COG': 5826.5458772495795, 'SD': 2750.0142483571017}


In [5]:
# MFCC: mel-frequency component coefficients
# how: signal > short frames > window (change slopes to increase energy of higher freq)
# > STFT > filterbanks (trianglular filters) > log(.) > decorrelation(inverse fourier)

mfcc_features = my_feature_generator._get_mfcc(sound,0,100)
print(mfcc_features)

[213.83727733  20.61894528  83.29904907   4.06891504  37.93732714
 -14.48091204   2.16565917   2.57373749   2.94791236  -5.63080092
   7.28615763   6.38696597]


In [6]:
# formant: frequencies of "sound bars"  - f1, f2, f3
formant_features = my_feature_generator._get_formant(sound,100,1000)
print(formant_features)

{'f1': [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], 'f2': [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], 'f3': [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]}


In [7]:
# pitch: human perceived frequency
pitch_features = my_feature_generator._get_pitch(sound,100,1000)
print(pitch_features)

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]


In [8]:
# intensity: power per unit carried by the soundwave
intensity_features = my_feature_generator._get_intensity(sound,100,1000)
print(intensity_features)

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]


In [9]:
# time difference between two consecutive periods
jitter_features = my_feature_generator._get_jitter(sound,100,1000)
print(jitter_features)

[nan, nan, nan, nan, nan]


In [10]:
# time difference between two consecutive peaks in amplitude
shimmer_features = my_feature_generator._get_shimmer(sound,100,1000)
print(shimmer_features)

[nan, nan, nan, nan, nan, nan]


In [11]:
# HNR: Harmonic to Noise Ratio
hnr_features = my_feature_generator._get_hnr(sound,100,1000)
print(hnr_features)

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]


In [12]:
import librosa
import numpy as np
from scipy.signal import hilbert

def librosa_feats(wv):
    y, sr = librosa.load(wv)
    # spectral centroid - where the center of spectrum energy
    cent = librosa.feature.spectral_centroid(y=y, sr=sr)[0]

    # bandwidth - freq range (high-low)
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)[0]

    # contrast - amplitude range
    S = np.abs(librosa.stft(y))
    contrast = librosa.feature.spectral_contrast(S=S, sr=sr)[0]

    # flatness - how much a sound is close to a pure tone
    flatness = librosa.feature.spectral_flatness(y=y)[0]

    # rolloff - frequency boundry between unvoiced and voiced
    # Approximate maximum frequencies with roll_percent=0.85 (default)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)[0]

    # zero-crossing - how much frequency changes acrosss 0-amp line
    zc = librosa.feature.zero_crossing_rate(y)[0]

    # rms: average sound intensity over time
    rms = librosa.feature.rms(y=y)[0]

    # envelope -hilbert envelope - an outline of amplitude change
    analytic_signal = hilbert(y)
    amplitude_envelope = np.abs(analytic_signal)
    return {'Centroid':cent, 'Bandwidth':spec_bw, 'Contrast':contrast, 'Flatness':flatness,
            'Rolloff':rolloff, 'Zero-Crossing':zc, 'RMS': rms, 'Envelope':amplitude_envelope}


In [13]:
result = librosa_feats("/content/harvard.wav")
print(result)

{'Centroid': array([5199.67804738, 5137.37377249, 5206.85589598, 5363.07262144,
       5238.38201193, 5239.57901179, 5343.93192288, 5268.30877635,
       5331.53365942, 5331.4505373 , 5361.90283999, 5300.07927631,
       5353.82464443, 5355.67911753, 5313.41954411, 5359.53374582,
       5275.67387856, 5304.18163013, 5320.56681029, 5142.70234302,
       5257.59686308, 5383.68951778, 5353.20972081, 5284.63951428,
       5365.62713267, 5410.04717516, 5333.9898304 , 5380.13366088,
       5436.51186595, 5237.52099147, 5201.41457506, 3383.34812325,
       4376.43270935, 4785.2153471 , 5274.01639492, 5218.15245938,
       4808.69229953, 2426.83811009, 1764.61585464, 1853.0081461 ,
       2139.12916149, 2354.69273236, 2792.78304862, 3079.08415044,
       3436.25046428, 3368.60892143, 2877.28641653, 2903.98584495,
       3040.06575806, 2801.86971716, 2358.03753962, 2141.49034097,
       1326.5199007 , 1291.21444844, 2188.7023349 , 1025.06650643,
        516.55742471,  557.90533067, 1592.2458717